# DTI-TK DTI analysis
n.bourke@imperial.ac.uk  
  
This notebook will run a DTI analysis following the TBSS pipeline, including DTI-TK registration.
Data structure follows BIDS format  
https://bids.neuroimaging.io/

------

## Setup environment
##### python cell
    1. enter project ID
    2. If this project doesn't exist in in the temporary space, create it
    3. Set important paths that will be used
    4. Make a setup script to save bash variables in

In [2]:
import os

project = "PTBI" 

directory = ("/rds/general/project/c3nl_djs_imaging_data/ephemeral/" + project + "/data/")
if not os.path.exists(directory):
    os.makedirs(directory)
    
raw = ("/rds/general/project/c3nl_djs_imaging_data/live/data/raw/" + project + "/")
source = ("/rds/general/project/c3nl_djs_imaging_data/live/data/sourcedata/")
workingDir = ("/rds/general/project/c3nl_djs_imaging_data/ephemeral/" + project + "/")
setup = (workingDir + "/dti_setup.sh" )


In [3]:
%%writefile $setup

$(: Project label) 
project="PTBI"

$(:  Where is the RAW directory?)
export raw=/rds/general/project/c3nl_djs_imaging_data/live/data/raw/${project};

$(: dependencies)
export dep=/rds/general/project/c3nl_shared/live/dependencies/
export templates=/rds/general/project/c3nl_djs_imaging_data/live/analysis/paeds/templates
export workingDir=/rds/general/project/c3nl_djs_imaging_data/ephemeral/${project}

dtitk_setup="export DTITK_ROOT=/apps/dti-tk/2.3.1/; export PATH=$PATH:$DTITK_ROOT/bin:$DTITK_ROOT/utilities:$DTITK_ROOT/scripts:$DTITK_ROOT/lib:$DTITK_ROOT/include" 
#echo ${PATH}

# Define modules    
fsl="module load fsl";
mricrogl="module load mricogl";


Overwriting /rds/general/project/c3nl_djs_imaging_data/ephemeral/PTBI//dti_setup.sh


In [9]:
%%bash -s "$setup"
export setup=$1;
source ${setup}
echo ${workingDir}

/rds/general/project/c3nl_djs_imaging_data/ephemeral/PTBI


### Copy data

Data valadation - check data is correctly pulled from raw to source data

In [ ]:
# import os
# import fnmatch

    
# for subject in os.listdir(raw):
#     #print(subject)  
#     subDir = (source + "sub-" + subject + "/")
#     for session in os.listdir(subDir):
#         in_file = (subDir + session + "/dwi/msdwi/sub-" + subject + "_" + session + "_msdwi.nii")

#         try:
#             f = open(in_file)
#             print(in_file + ' File exists')
#             f.close()
#         except FileNotFoundError:
#             print(in_file + ' WARNING: File does not exist')
            

### Stage 1: File management - create an organised data directory
#### Pulling from BIDS sourcedata structure 

In [7]:
%%bash -s "$setup"
export setup=$1;
source ${setup}
echo ${workingDir}
# ----------------

sourceDir=/rds/general/project/c3nl_djs_imaging_data/live/data/sourcedata/

# setup log dir
if [ ! -d ${workingDir}/commandLogs/ ]; then
    echo "Making log dir!"
    mkdir -p ${workingDir}/commandLogs/
fi


echo -n "" > ${workingDir}/commandLogs/dwiMissingData.txt;

for ii in `ls -d ${raw}/*`; 
do
    sub=`basename $ii`
    for ses in `ls ${sourceDir}/sub-${sub}`;
    do
        for session in `ls ${sourceDir}/sub-${sub}/`;
        do
            if [ ! -d ${sourceDir}/sub-${sub}/${session}/dwi/dti/ ]; then 
                echo "DTI data not found for ${ses}" >> ${workingDir}/commandLogs/msdwiMissingData.txt;
            else
                mkdir -p ${workingDir}/data/sub-${sub}/${ses}/dwi/dti
                #mkdir -p ${workingDir}/data/sub-${sub}/${ses}/anat/T1w
                cp ${sourceDir}/sub-${sub}/${ses}/dwi/dti/*  ${workingDir}/data/sub-${sub}/${ses}/dwi/dti/;
                #cp ${source}/sub-${subject}/${session}/anat/T1w/*  ${workingDir}/data/sub-${sub}/${ses}/anat/T1w/;
            fi
        done
    done    
done

/rds/general/project/c3nl_djs_imaging_data/ephemeral/PTBI


In [8]:
%%bash -s "$setup"
export setup=$1;
source ${setup}
echo ${workingDir}

# module load fsl
#----------------------------

# setup log dir
if [ ! -d ${workingDir}/commandLogs/ ]; then
    echo "Making log dir!"
    mkdir ${workingDir}/commandLogs/
fi
#

# Make a list of all DWI scans
echo -n "" > ${workingDir}/dti_list.txt
for s in `ls -d ${workingDir}/data/*`;
    do
    subj=`basename ${s}`
    for ses in `ls ${workingDir}/data/${subj}`; 
        do 
        echo ${workingDir}/data/${subj}/${ses}/dwi/dti/${subj}_${ses}_dti.nii >> ${workingDir}/dti_list.txt 
    done
done


/rds/general/project/c3nl_djs_imaging_data/ephemeral/PTBI


#### Checking the data

### Stage 2: Pre-processing diffusion data

### Correct for eddy distortion and motion in the data

In [16]:
%%bash -s "$setup"
export setup=$1;
source ${setup}
echo ${workingDir}
#----------------

echo -n "" > ${workingDir}/commandLogs/ec_correct.txt
job=${workingDir}/commandLogs/ec_correct.txt

for s in `ls -d ${workingDir}/data/*`;
    do
    subj=`basename ${s}`
    for ses in `ls ${workingDir}/data/${subj}`; 
        do         
        path=${workingDir}/data/${subj}/${ses}/dwi/dti
        dwi=`ls ${path}/*dti.ni*`
        bvec=`ls ${path}/*.bvec`
        bval=`ls ${path}/*.bval`
    
        echo "${fsl}; eddy_correct ${dwi} ${path}/data.nii.gz 0; fslroi ${path}/data.nii.gz ${path}/nodif.nii.gz 0 1; fdt_rotate_bvecs ${bvec} ${path}/rotated_bvec ${path}/data.ecclog; bet ${path}/nodif.nii.gz ${path}/nodif_brain -f 0.2 -g  0 -m; mv ${bval} ${path}/${ses}_dti.bval; dtifit --data=${path}/data.nii.gz --out=${path}/dti --mask=${path}/nodif_brain_mask --bvecs=${path}/rotated_bvec --bvals=${path}/${ses}_dti.bval -w"  >> ${job}  

    done    
done

    # Run job
    ${dep}/hpcSubmit ${job} 04:00:00 3 6Gb
    echo ""; echo "***"; echo ""; echo "Submitted commands:"
    head ${job}
 

/rds/general/project/c3nl_djs_imaging_data/ephemeral/PTBI
input is = /rds/general/project/c3nl_djs_imaging_data/ephemeral/PTBI/commandLogs/ec_correct.txt
Walltime = 04:00:00
Number of CPUs = 3
Memory = 6Gb
Array jobs submitted: 60
Job submitted: Mon  6 Apr 10:06:46 BST 2020
1280195[].pbs

***

Submitted commands:
module load fsl; eddy_correct  /rds/general/project/c3nl_djs_imaging_data/ephemeral/PTBI/data/sub-CIF0537/ses-2017-03-27-PILOT_AMR/dwi/dti/data.nii.gz 0; fslroi /rds/general/project/c3nl_djs_imaging_data/ephemeral/PTBI/data/sub-CIF0537/ses-2017-03-27-PILOT_AMR/dwi/dti/data.nii.gz /rds/general/project/c3nl_djs_imaging_data/ephemeral/PTBI/data/sub-CIF0537/ses-2017-03-27-PILOT_AMR/dwi/dti/nodif.nii.gz 0 1; fdt_rotate_bvecs  /rds/general/project/c3nl_djs_imaging_data/ephemeral/PTBI/data/sub-CIF0537/ses-2017-03-27-PILOT_AMR/dwi/dti/rotated_bvec /rds/general/project/c3nl_djs_imaging_data/ephemeral/PTBI/data/sub-CIF0537/ses-2017-03-27-PILOT_AMR/dwi/dti/data.ecclog; bet /rds/general/p

ls: cannot access /rds/general/project/c3nl_djs_imaging_data/ephemeral/PTBI/data/sub-CIF0537/ses-2017-03-27-PILOT_AMR/dwi/dti/*dti.ni*: No such file or directory
ls: cannot access /rds/general/project/c3nl_djs_imaging_data/ephemeral/PTBI/data/sub-CIF0537/ses-2017-03-27-PILOT_AMR/dwi/dti/*.bvec: No such file or directory
ls: cannot access /rds/general/project/c3nl_djs_imaging_data/ephemeral/PTBI/data/sub-CIF0537/ses-2017-03-27-PILOT_AMR/dwi/dti/*.bval: No such file or directory


### Check the state of the submitted cluster jobs

## Running DTI-TK

The Diffusion Tensor Imaging Toolkit (DTI-TK) is an advanced toolkit for optimising DTI analysis. Further information can be found at: http://dti-tk.sourceforge.net/pmwiki/pmwiki.php. Specific information about the tools and commands used in this pipeline can be found in the "Documentation -> List of tools" section of the website. 

This process may take a while, even when submitting through `qsub` or an array job on the HPC.
As such it is required to break up into steps (template building done as seperate job).

These steps utilise some diffusion imaging templates for the purposes of registering your study data. If you wish to create a study-specific template using DTITK, relevant information can be found here: http://dti-tk.sourceforge.net/pmwiki/pmwiki.php?n=Documentation.Registration

**First, convert fsl format data to dtitk format.** *Tip: If you are processing data from separate visits, you could add a visit-specific sub-directory to the code below, so your data is organised correctly.*



In [10]:
%%bash -s "$setup"
export setup=$1;
source ${setup}
echo ${workingDir}
#----------------

echo -n "" > ${workingDir}/commandLogs/fsl2tk.txt
job=${workingDir}/commandLogs/fsl2tk.txt

for s in `ls -d ${workingDir}/data/*`;
    do
    subj=`basename ${s}`
    for ses in `ls ${workingDir}/data/${subj}`; 
        do
        echo "${fsl}; fsl_to_dtitk ${workingDir}/data/${subj}/${ses}/dwi/dti/dti" >> ${job}
    done    
done

    # Run job
    ${dep}/hpcSubmit ${job} 01:00:00 1 6Gb
    echo ""; echo "***"; echo ""; echo "Submitted commands:"
    head ${job}     
    

/rds/general/project/c3nl_djs_imaging_data/ephemeral/PTBI
input is = /rds/general/project/c3nl_djs_imaging_data/ephemeral/PTBI/commandLogs/fsl2tk.txt
Walltime = 01:00:00
Number of CPUs = 1
Memory = 6Gb
Array jobs submitted: 60
Job submitted: Mon  6 Apr 10:40:25 BST 2020
1280300[].pbs

***

Submitted commands:
module load fsl; fsl_to_dtitk /rds/general/project/c3nl_djs_imaging_data/ephemeral/PTBI/data/sub-CIF0537/ses-2017-03-27-PILOT_AMR/dwi/dti/dti
module load fsl; fsl_to_dtitk /rds/general/project/c3nl_djs_imaging_data/ephemeral/PTBI/data/sub-CIF1638/ses-2017-03-30-PTBI001/dwi/dti/dti
module load fsl; fsl_to_dtitk /rds/general/project/c3nl_djs_imaging_data/ephemeral/PTBI/data/sub-CIF1702/ses-2017-05-31-PTBI003/dwi/dti/dti
module load fsl; fsl_to_dtitk /rds/general/project/c3nl_djs_imaging_data/ephemeral/PTBI/data/sub-CIF1703/ses-2017-05-31-PTBI101/dwi/dti/dti
module load fsl; fsl_to_dtitk /rds/general/project/c3nl_djs_imaging_data/ephemeral/PTBI/data/sub-CIF1704/ses-2017-06-01-PTBI002

#### Build a list of paths for data to process

In [13]:
%%bash -s "$setup"
export setup=$1;
source ${setup}
echo ${workingDir}
#----------------

echo -n "" > ${workingDir}/dtitk_subj.txt

for s in `ls -d ${workingDir}/data/*`;
    do
    subj=`basename ${s}`
    for ses in `ls ${workingDir}/data/${subj}`; 
        do
        ls ${workingDir}/data/${subj}/${ses}/dwi/dti/*_dtitk.nii.gz >> ${workingDir}/dtitk_subj.txt
    done    
done


/rds/general/project/c3nl_djs_imaging_data/ephemeral/PTBI


ls: cannot access /rds/general/project/c3nl_djs_imaging_data/ephemeral/PTBI/data/sub-CIF0537/ses-2017-03-27-PILOT_AMR/dwi/dti/*_dtitk.nii.gz: No such file or directory


## Step 1. Initiate bootstrapping to create sample template

In [16]:
%%bash -s "$setup"
export setup=$1;
source ${setup}
echo ${workingDir}
#----------------

echo -n "" > ${workingDir}/commandLogs/boot.txt;
job=${workingDir}/commandLogs/boot.txt;

mkdir ${workingDir}/DTITK
DTITK_DATA_DIR=${workingDir}/DTITK


# setting up user environment
echo "${fsl}" >> ${job};
echo "${mricrogl}" >> ${job};
echo "${dtitk_setup}" >> ${job};
echo "cd ${DTITK_DATA_DIR}" >> ${job};

# Initiating bootstrapping
echo "dti_template_bootstrap ${templates}/ixi_aging_template.nii.gz ${workingDir}/dtitk_subj.txt;" >> ${job}; 
echo 'echo "Complete-now performing affine registration";' >> ${job};
echo "dti_affine_population mean_initial.nii.gz ${workingDir}/dtitk_subj.txt EDS 3;" >> ${job};
echo 'echo "Complete";' >> ${job};
echo "TVtool -in mean_affine3.nii.gz -tr;" >> ${job};
echo 'echo "Creating mask for diffeomorphic registration";' >> ${job};
echo "BinaryThresholdImageFilter mean_affine3_tr.nii.gz mask.nii.gz 0.01 100 1 0;" >> ${job};
   
# Run job
qsub -lselect=1:ncpus=1:mem=6Gb -l walltime=52:00:00 ${job}   
echo ""; echo "***"; echo ""; echo "Submitted commands:"
head ${job}  

/rds/general/project/c3nl_djs_imaging_data/ephemeral/PTBI
1280448.pbs

***

Submitted commands:
module load fsl


cd /rds/general/project/c3nl_djs_imaging_data/ephemeral/PTBI/DTITK
dti_template_bootstrap /rds/general/project/c3nl_djs_imaging_data/live/analysis/paeds/templates/ixi_aging_template.nii.gz /rds/general/project/c3nl_djs_imaging_data/ephemeral/PTBI/dtitk_subj.txt;
echo "Complete-now performing affine registration";
dti_affine_population mean_initial.nii.gz /rds/general/project/c3nl_djs_imaging_data/ephemeral/PTBI/dtitk_subj.txt EDS 3;
echo "Complete";
TVtool -in mean_affine3.nii.gz -tr;
echo "Creating mask for diffeomorphic registration";


mkdir: cannot create directory ‘/rds/general/project/c3nl_djs_imaging_data/ephemeral/PTBI/DTITK’: File exists


## Step 2: Generate registration to template matricies

In [5]:
%%bash -s "$setup"
export setup=$1;
source ${setup}
echo ${workingDir}
#----------------

echo -n "" > ${workingDir}/commandLogs/DTITKJob.txt;
job=${workingDir}/commandLogs/DTITKJob.txt;
DTITK_DATA_DIR=${workingDir}/DTITK

# setting up user environment
echo "${fsl}" >> ${job};
echo "${mricrogl}" >> ${job};
echo "${dtitk_setup}" >> ${job};
echo "cd ${DTITK_DATA_DIR}" >> ${job};

# Running registration
echo 'echo "Running diffeomorphic registration";' >> ${job};
echo "dti_diffeomorphic_population mean_affine3.nii.gz ${workingDir}/dtitk_subj_aff.txt mask.nii.gz 0.002;" >> ${job};
echo 'echo "Complete";' >> ${job};

echo 'echo "Making images 1mm isotropic";' >> ${job};
echo "dti_warp_to_template_group ${workingDir}/dtitk_subj.txt mean_diffeomorphic_initial6.nii.gz 1 1 1;" >> ${job};

echo 'echo "Combining the affine and diffeomorphic registrations";' >> ${job};
echo "for f in \`cat ${workingDir}/dtitk_subj.txt\`; do echo \${f/.nii.gz/}; done > ${workingDir}/DTI_subjects_combined.txt;" >> ${job};
echo "for f in \`cat ${workingDir}/DTI_subjects_combined.txt\`; do dfRightComposeAffine -aff \${f}.aff -df \${f}_aff_diffeo.df.nii.gz -out \${f}_combined.df.nii.gz; done;" >> ${job}; 

echo 'echo "Rigid registration to IITmeantensor256-MNI space";' >> ${job};
echo "dti_rigid_reg ${templates}/IITmean_tensor_256.nii mean_diffeomorphic_initial6.nii.gz EDS 4 4 4 0.001;" >> ${job};

echo 'echo "Affine registration to IITmeantensor256";' >> ${job};
echo "dti_affine_reg ${templates}/IITmean_tensor_256.nii mean_diffeomorphic_initial6.nii.gz EDS 4 4 4 0.001 1;" >> ${job};

echo 'echo "Diffeomorphic registration to IITmeantensor256";' >> ${job};
echo "dti_diffeomorphic_reg ${templates}/IITmean_tensor_256.nii mean_diffeomorphic_initial6_aff.nii.gz ${templates}/IITmean_tensor_mask_256.nii.gz 1 6 0.002;" >> ${job};
echo "dfRightComposeAffine -aff mean_diffeomorphic_initial6.aff -df mean_diffeomorphic_initial6_aff_diffeo.df.nii.gz -out mean_combined.df.nii.gz;" >> ${job};


# Run job
qsub -lselect=1:ncpus=1:mem=6Gb -l walltime=52:00:00 ${job}   
echo ""; echo "***"; echo ""; echo "Submitted commands:"
head ${job}  

/rds/general/project/c3nl_djs_imaging_data/ephemeral/PTBI
1289593.pbs

***

Submitted commands:
module load fsl
module load mricogl
export DTITK_ROOT=/apps/dti-tk/2.3.1/; export PATH=/apps/jupyterhub/latest:/bin:/usr/bin:/sbin:/usr/sbin:/opt/pbs/default/bin:/usr/local/bin:/apps/dti-tk/2.3.1//bin:/apps/dti-tk/2.3.1//utilities:/apps/dti-tk/2.3.1//scripts:/apps/dti-tk/2.3.1//lib:/apps/dti-tk/2.3.1//include
cd /rds/general/project/c3nl_djs_imaging_data/ephemeral/PTBI/DTITK
echo "Diffeomorphic registration to IITmeantensor256";
dti_diffeomorphic_reg /rds/general/project/c3nl_djs_imaging_data/live/analysis/paeds/templates/IITmean_tensor_256.nii mean_diffeomorphic_initial6_aff.nii.gz /rds/general/project/c3nl_djs_imaging_data/live/analysis/paeds/templates/IITmean_tensor_mask_256.nii.gz 1 6 0.002;
dfRightComposeAffine -aff mean_diffeomorphic_initial6.aff -df mean_diffeomorphic_initial6_aff_diffeo.df.nii.gz -out mean_combined.df.nii.gz;


## Step 3. Moving images to standard space using the translation matricies derived above

In [6]:
%%bash -s "$setup"
export setup=$1;
source ${setup}
echo ${workingDir}
#----------------

echo -n "" > ${workingDir}/commandLogs/DTITK_reg_Job.txt;
job=${workingDir}/commandLogs/DTITK_reg_Job.txt;
DTITK_DATA_DIR=${workingDir}/DTITK

# setting up user environment
# echo "${fsl}" >> ${job};
# echo "${mricrogl}" >> ${job};
# echo "${dtitk_setup}" >> ${job};
# echo "cd ${DTITK_DATA_DIR}" >> ${job};


# Running registration
# (take the raw data and use the combination of translations to move to standard space in a single step)

for f in `cat ${workingDir}/DTI_subjects_combined.txt`; 
    do 
    echo "${fsl}; ${mricrogl}; ${dtitk_setup}; cd ${DTITK_DATA_DIR}; dfComposition -df2 ${f}_combined.df.nii.gz -df1 mean_combined.df.nii.gz -out ${f}_to_standard.df.nii.gz; deformationSymTensor3DVolume -in ${f}.nii.gz -trans ${f}_to_standard.df.nii.gz -target ${templates}/IITmean_tensor_256.nii -out ${f}_to_standard.nii.gz;" >> ${job};
done


# echo 'echo "Transforming subjects from native to standard space";' >> ${job};
# echo "for f in \`cat ${workingDir}/DTI_subjects_combined.txt\`; do dfComposition -df2 \${f}_combined.df.nii.gz -df1 mean_combined.df.nii.gz -out \${f}_to_standard.df.nii.gz; done;" >> ${job};
# echo "for f in \`cat ${workingDir}/DTI_subjects_combined.txt\`; do deformationSymTensor3DVolume -in \${f}.nii.gz -trans \${f}_to_standard.df.nii.gz -target ${templates}/IITmean_tensor_256.nii -out \${f}_to_standard.nii.gz; done;" >> ${job}; 


    # Run job
    ${dep}/hpcSubmit ${job} 06:00:00 3 12Gb
    echo ""; echo "***"; echo ""; echo "Submitted commands:"
    head ${job}
    

/rds/general/project/c3nl_djs_imaging_data/ephemeral/PTBI
input is = /rds/general/project/c3nl_djs_imaging_data/ephemeral/PTBI/commandLogs/DTITK_reg_Job.txt
Walltime = 06:00:00
Number of CPUs = 3
Memory = 12Gb
Array jobs submitted: 59
Job submitted: Wed  8 Apr 13:00:07 BST 2020
1289770[].pbs

***

Submitted commands:
module load fsl; module load mricogl; export DTITK_ROOT=/apps/dti-tk/2.3.1/; export PATH=/apps/jupyterhub/latest:/bin:/usr/bin:/sbin:/usr/sbin:/opt/pbs/default/bin:/usr/local/bin:/apps/dti-tk/2.3.1//bin:/apps/dti-tk/2.3.1//utilities:/apps/dti-tk/2.3.1//scripts:/apps/dti-tk/2.3.1//lib:/apps/dti-tk/2.3.1//include; cd /rds/general/project/c3nl_djs_imaging_data/ephemeral/PTBI/DTITK; dfComposition -df2 /rds/general/project/c3nl_djs_imaging_data/ephemeral/PTBI/data/sub-CIF1638/ses-2017-03-30-PTBI001/dwi/dti/dti_dtitk_combined.df.nii.gz -df1 mean_combined.df.nii.gz -out /rds/general/project/c3nl_djs_imaging_data/ephemeral/PTBI/data/sub-CIF1638/ses-2017-03-30-PTBI001/dwi/dti/dti_d

# TBSS pipeline (with DTITK registered data)

In [4]:
%%bash -s "$setup"
export setup=$1;
source ${setup}
echo ${workingDir}
#----------------

echo -n "" > ${workingDir}/commandLogs/TBSS_Job.txt;
job=${workingDir}/commandLogs/TBSS_Job.txt;
DTITK_DATA_DIR=${workingDir}/DTITK


echo -n "" > ${workingDir}/DTI_subjs_normalised256.txt
echo -n "" > ${workingDir}/fa_normalised256.txt
for s in `ls -d ${workingDir}/data/*`;
    do
    subj=`basename ${s}`
    for ses in `ls ${workingDir}/data/${subj}`; 
        do
        ls ${workingDir}/data/${subj}/${ses}/dwi/dti/dti_dtitk_to_standard.nii.gz >> ${workingDir}/DTI_subjs_normalised256.txt
        ls ${workingDir}/data/${subj}/${ses}/dwi/dti/dti_dtitk_to_standard_fa.nii.gz >> ${workingDir}/fa_normalised256.txt
    done    
    done    
done

# setting up user environment
echo "${mricrogl}" >> ${job};
echo "${dtitk_setup}" >> ${job};
echo "${fsl}" >> ${job}; # needs to be loaded after messing with paths in dtitk_setup
echo "cd ${DTITK_DATA_DIR}" >> ${job};

# echo 'echo "Creating 4D file of all subjects registrations to standard space for QC";' >> ${job};
# echo "fslmerge -t all_subjs_to_standard_tensor.nii.gz \`cat ${workingDir}/DTI_subjs_normalised256.txt\`;" >> ${job};

# echo 'echo "Creating mean tensor image for subjects";' >> ${job};
# #echo "for f in *_to_standard.nii.gz; do echo \${f}; done > DTI_subjs_normalised256.txt;" >> ${job};
# echo "TVMean -in ${workingDir}/DTI_subjs_normalised256.txt -out mean_final_high_res.nii.gz;" >> ${job};

# echo 'echo "Creating mean FA image for all subjects";' >> ${job};
# echo "TVtool -in mean_final_high_res.nii.gz -fa;" >> ${job};
# echo "cp mean_final_high_res_fa.nii.gz mean_FA.nii.gz;" >> ${job};

# # For some reason fsl wasnt being loaded and broke at this point. 
# echo "tbss_skeleton -i mean_FA.nii.gz -o mean_FA_skeleton;" >> ${job};
# echo 'echo "Creating individuals FA maps normalised to standard space";' >> ${job};
# echo "for f in \`cat ${workingDir}/DTI_subjs_normalised256.txt\`; do TVtool -in \${f} -fa;TVtool -in \${f} -tr; done;" >> ${job};
# echo 'echo "Creating MD maps normalised to standard space";' >> ${job};
# echo "for f in \`cat ${workingDir}/DTI_subjs_normalised256.txt\`; do  MDsubj=\${f%_to_standard.nii.gz}; fslmaths \${MDsubj}_to_standard_tr.nii.gz -div 3 \${MDsubj}_to_standard_md.nii.gz; done;" >> ${job};


echo "fslmerge -t all_FA.nii.gz \`cat ${workingDir}/fa_normalised256.txt\`;" >> ${job};
echo 'echo "Creating TBSS directories for TBSS processing step 4 and statistics";' >> ${job};
echo "mkdir TBSS;" >> ${job};
echo "mkdir TBSS/FA;" >> ${job};
echo "mkdir TBSS/stats;" >> ${job};
echo "cp ${templates}/IITmean_tensor_256.nii TBSS/FA/target.nii.gz;" >> ${job};
echo "cp all_FA.nii.gz TBSS/stats/;" >> ${job};
echo "cp mean_FA* TBSS/stats/;" >> ${job};
echo "fslmaths all_FA -max 0 -Tmin -bin mean_FA_mask -odt char;" >> ${job};
echo "cp mean_FA_mask.nii.gz TBSS/stats/;" >> ${job};
echo "cd TBSS/;" >> ${job};
echo "tbss_4_prestats 0.2;" >> ${job};
echo 'echo "Done - Ready for TBSS or single subject statistics"' >> ${job};

# Run job
qsub -lselect=1:ncpus=1:mem=44Gb -l walltime=32:00:00 ${job}   
echo ""; echo "***"; echo ""; echo "Submitted commands:"
head ${job} 

/rds/general/project/c3nl_djs_imaging_data/ephemeral/PTBI
1293456.pbs

***

Submitted commands:
module load mricogl
export DTITK_ROOT=/apps/dti-tk/2.3.1/; export PATH=/apps/jupyterhub/latest:/bin:/usr/bin:/sbin:/usr/sbin:/opt/pbs/default/bin:/usr/local/bin:/apps/dti-tk/2.3.1//bin:/apps/dti-tk/2.3.1//utilities:/apps/dti-tk/2.3.1//scripts:/apps/dti-tk/2.3.1//lib:/apps/dti-tk/2.3.1//include
module load fsl
cd /rds/general/project/c3nl_djs_imaging_data/ephemeral/PTBI/DTITK
cd TBSS/;
tbss_4_prestats 0.2;
echo "Done - Ready for TBSS or single subject statistics"


ls: cannot access /rds/general/project/c3nl_djs_imaging_data/ephemeral/PTBI/data/sub-CIF0537/ses-2017-03-27-PILOT_AMR/dwi/dti/dti_dtitk_to_standard.nii.gz: No such file or directory


## ROI
These are select examples from social papers 

In [2]:
%%bash -s "$setup"
export setup=$1;
source $setup
echo $wd
#----------------

cd /rds/general/user/nbourke/home/templates/social_ROI

## TPJ
# MNI coordinates = 55 -52 15
make_sphere_roi ~/templates/MNI152_T1_1mm.nii.gz 35 74 87 5

## ACCg
# MNI coordinates = 2 24 17.5
${dep}/make_sphere_roi ~/templates/MNI152_T1_1mm.nii.gz 88 150 89 5

## dmpfc
# MNI coordinates = 2.5 40 41
${dep}/make_sphere_roi ~/templates/MNI152_T1_1mm.nii.gz 87 166 113 5

## sgACC
# MNI coordinates = -3.5 8 -11
${dep}/make_sphere_roi ~/templates/MNI152_T1_1mm.nii.gz 93 134 61 5


******************************
Input image: /rds/general/user/nbourke/home/templates/MNI152_T1_1mm.nii.gz 
ROI coordinates: 88 150 89 
ROI size: 5 
Done! The output file is called 'mask_sphere_88-150-89_5mm' 
 
******************************
Input image: /rds/general/user/nbourke/home/templates/MNI152_T1_1mm.nii.gz 
ROI coordinates: 87 166 113 
ROI size: 5 
Done! The output file is called 'mask_sphere_87-166-113_5mm' 
 
******************************
Input image: /rds/general/user/nbourke/home/templates/MNI152_T1_1mm.nii.gz 
ROI coordinates: 93 134 61 
ROI size: 5 
Done! The output file is called 'mask_sphere_93-134-61_5mm' 
 


bash: line 10: make_sphere_roi: command not found
 /home/nbourke/anaconda3/envs/py27/bin /apps/gcc/6.2.0/ /opt/ibutils/bin /rds/general/user/nbourke/home/anaconda3/bin /apps/mrtrix/3.0/bin /apps/ants/2015-02-23/bin/bin /apps/gcc/6.2.0/bin /usr/local/sbin /opt/pbs/bin /usr/lib64/qt-3.3/bin /apps/ants/2015-02-23/bin/ /rds/general/user/nbourke/home/perl5/bin /apps/anaconda3/4.5.12/install
 /home/nbourke/anaconda3/envs/py27/bin /apps/gcc/6.2.0/ /opt/ibutils/bin /rds/general/user/nbourke/home/anaconda3/bin /apps/mrtrix/3.0/bin /apps/ants/2015-02-23/bin/bin /apps/gcc/6.2.0/bin /usr/local/sbin /opt/pbs/bin /usr/lib64/qt-3.3/bin /apps/ants/2015-02-23/bin/ /rds/general/user/nbourke/home/perl5/bin /apps/anaconda3/4.5.12/install
 /home/nbourke/anaconda3/envs/py27/bin /apps/gcc/6.2.0/ /opt/ibutils/bin /rds/general/user/nbourke/home/anaconda3/bin /apps/mrtrix/3.0/bin /apps/ants/2015-02-23/bin/bin /apps/gcc/6.2.0/bin /usr/local/sbin /opt/pbs/bin /usr/lib64/qt-3.3/bin /apps/ants/2015-02-23/bin/ /rds/

In [ ]:
#https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/TBSS/UserGuide

cd ../stats
design_ttest2 design 7 11

randomise -i all_FA_skeletonised -o tbss -m mean_FA_skeleton_mask -d design.mat -t design.con -n 500 --T2

fslview $FSLDIR/data/standard/MNI152_T1_1mm mean_FA_skeleton -l Green -b 0.2,0.8 tbss_tstat1 -l Red-Yellow -b 3,6 tbss_tstat2 -l Blue-Lightblue -b 3,6

## Save out summary measures for tracts

In [11]:
%%bash -s "$setup"
export setup=$1;
source $setup
echo ${workingDir}
${fsl}
#----------------

dataDir=${workingDir}/DTITK/TBSS
tractDIR=~/templates/Corrected_Tracts_256
        
mkdir ${dataDir}/tractStats

# Find the tract masks
cd ${tractDIR}
TRACTS=`ls *.gz`

# Nested for loop - for each mask and each metric do fslstats
for i in $TRACTS; 
do
    #j=$(echo ${i} | cut -d '_' -f2-)
    k=$(echo ${i} | cut -d '.' -f1)
    echo $k > ${dataDir}/tractStats/fa_MNI_${k}.txt
    
    fslstats -t ${dataDir}/stats/all_FA.nii.gz -k ${tractDIR}/$i -M >> ${dataDir}/tractStats/fa_MNI_${k}.txt

done 

echo "" >  ${dataDir}/tractStats/aaa.txt
cat ${workingDir}/fa_normalised256 >> ${dataDir}/tractStats/aaa.txt
echo "fa_MNI_skeleton" > ${dataDir}/tractStats/MNI_FA_skeleton.txt
fslstats -t ${dataDir}/stats/all_FA.nii.gz -k ${dataDir}/stats/mean_FA_skeleton_mask.nii.gz -M >> ${dataDir}/tractStats/MNI_FA_skeleton.txt

# paste -d , `ls` >> ../fa_tractStats.csv


/rds/general/project/c3nl_djs_imaging_data/ephemeral/PTBI


 /home/nbourke/anaconda3/envs/py27/bin /apps/gcc/6.2.0/ /opt/ibutils/bin /rds/general/user/nbourke/home/anaconda3/bin /apps/mrtrix/3.0/bin /apps/ants/2015-02-23/bin/bin /apps/gcc/6.2.0/bin /usr/local/sbin /opt/pbs/bin /usr/lib64/qt-3.3/bin /apps/ants/2015-02-23/bin/ /rds/general/user/nbourke/home/perl5/bin /apps/anaconda3/4.5.12/install
cat: /rds/general/project/c3nl_djs_imaging_data/ephemeral/PTBI/fa_normalised256: No such file or directory


In [ ]:

for ii in `cat dtitk_subj.txt`; 
    do
    A="$(echo ${ii} | cut -d'/' -f10)"
    echo $A
done
    